In [1]:
import os
from glob import glob
from collections import OrderedDict, defaultdict
import math
import json

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from torchvision.datasets import CIFAR10

In [2]:
def entropy(p):
    nats = -p * np.log(p, where=p>0.0)
    return nats.sum()

def get_dist(targets, num_classes:int=10):
    freq = np.zeros(num_classes)
    uniques, cnts = np.unique(targets, return_counts=True)
    for u, c in zip(uniques, cnts):
        freq[u] = c
    freq = freq / sum(cnts)
    return freq

In [30]:
EXP_PATH = 'saved/c10_v16_maxent'

result_files = sorted(glob(os.path.join(EXP_PATH, "*", "results.json")))
config_files = sorted(glob(os.path.join(EXP_PATH, "*", "config.json")))

print(f"Found {len(result_files)} experiments.")

Found 1 experiments.


In [31]:
run_names = []
all_results = []
all_configs = []

for r, c in zip(result_files, config_files):
    assert os.path.split(r)[0] == os.path.split(c)[0]
    with open(c, 'r') as f:
        config = json.load(f)
        run_names.append(config['run_name'])
        all_configs.append(config)
    with open(r, 'r') as f:
        all_results.append(json.load(f))

In [32]:
cifar10 = CIFAR10('/opt/datasets/cifar10', train=True)
targets = np.asarray(cifar10.targets)

result_dict = OrderedDict()
for run_name, result in zip(run_names, all_results):
    all_queried_ids = []
    all_metrics = defaultdict(list)
    for ep in result:
        ep_idx = ep['episode']

        # Typo
        try:
            queried_ids = ep['episode/indices']
        except:
            queried_ids = ep['episode/indicies']
        all_queried_ids.extend(queried_ids)
        
        all_metrics['entropy'].append(entropy(get_dist(targets[queried_ids], 10)))
        all_metrics['overall_entropy'].append(entropy(get_dist(targets[all_queried_ids], 10)))
        
        if ep_idx == 0:
            continue
        
        for k, v in ep.items():
            if isinstance(v, float):
                all_metrics[k].append(v)

    min_len = 100
    for k, v in all_metrics.items():
        if len(v) < min_len:
            min_len = len(v)
    all_metrics = {k: v[:min_len] for k, v in all_metrics.items()}
    
    result_dict[run_name] = all_metrics

In [33]:
print(all_configs[0])

{'file': 'configs/cifar10_vgg16.json', 'run_name': 'cifar10_snapshot_20220919_174546', 'save_path': 'saved/c10_v16_maxent/cifar10_snapshot_20220919_174546', 'dataset_name': 'cifar10', 'dataset_path': '/opt/datasets', 'seed': 42, 'arch': 'vgg16', 'disable_tqdm': False, 'resume_from': None, 'learning_rate': 0.001, 'weight_decay': 0.01, 'momentum': 0.9, 'batch_size': 64, 'num_epochs': 200, 'optimizer_type': 'sgd', 'lr_scheduler_type': 'constant', 'lr_scheduler_param': 10.0, 'num_workers': 4, 'log_every': 10, 'eval_every': 10, 'swa_start': 100, 'swa_anneal_epochs': 50, 'swa_lr_multiplier': 10.0, 'swa_scheduler_type': 'constant', 'start_swa_at_end': True, 'num_episodes': 30, 'num_ensembles': 5, 'query_size': 500, 'query_type': 'maxentropy', 'init_query_size': 500, 'init_query_type': 'random', 'eval_query_size': 500, 'eval_query_type': 'random', 'eval_size': 500}


In [34]:
pd.DataFrame(result_dict[run_names[0]]).T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
entropy,2.291243,2.248366,2.276165,2.183876,2.121027,2.186897,2.228213,2.157343,2.207487,2.184636,2.074640,1.901184,1.993374,1.937773,2.198380,1.866092,2.176153,2.208813,2.195562,2.091958
overall_entropy,2.291243,2.284166,2.293520,2.282068,2.289090,2.287332,2.285109,2.279786,2.283619,2.279585,2.285705,2.279408,2.286240,2.279615,2.282042,2.279221,2.278855,2.277678,2.278527,2.280042
eval/acc,0.402000,0.332000,0.390000,0.528000,0.480000,0.508000,0.522000,0.538000,0.554000,0.530000,0.576000,0.628000,0.668000,0.572000,0.684000,0.680000,0.620000,0.704000,0.704000,0.760000
eval/max_acc,0.402000,0.430000,0.490000,0.528000,0.548000,0.554000,0.576000,0.606000,0.626000,0.626000,0.628000,0.644000,0.680000,0.680000,0.684000,0.682000,0.668000,0.704000,0.704000,0.760000
test/swa_acc,0.449200,0.515300,0.584400,0.601200,0.618500,0.664400,0.682100,0.701500,0.725600,0.736900,0.770600,0.774000,0.782300,0.789400,0.808200,0.809100,0.819900,0.834700,0.827000,0.841700
test/swa_nll,2.312959,1.941686,1.630996,1.529135,1.299620,1.194669,1.065690,1.001198,0.886611,0.854641,0.716440,0.691743,0.674876,0.641045,0.584050,0.577012,0.559762,0.518876,0.532278,0.510366
test/swa_ece,0.342425,0.293719,0.243289,0.231784,0.186440,0.173881,0.148843,0.137520,0.107079,0.104168,0.054092,0.047032,0.036750,0.025469,0.016743,0.014856,0.013409,0.017686,0.017217,0.037530
test/swa_top5,0.891900,0.909900,0.929100,0.933600,0.948400,0.953300,0.964900,0.969400,0.969000,0.972700,0.977200,0.982100,0.977400,0.981100,0.987300,0.984400,0.986900,0.989400,0.988200,0.988800
ens/acc,0.446400,0.471900,0.535300,0.583700,0.628700,0.641700,0.640300,0.684200,0.710500,0.669900,0.711000,0.712200,0.739400,0.767100,0.743600,0.765200,0.793600,0.783700,0.797400,0.806900
ens/nll,1.871005,1.615373,1.380093,1.230280,1.061660,1.046766,1.057427,0.933162,0.858658,0.972901,0.861337,0.830404,0.802452,0.728640,0.778188,0.707454,0.667927,0.672652,0.645757,0.610079


In [39]:
METRICS = ['test/swa_acc', 'test/swa_nll', 'test/swa_ece']

all_dfs = []
for run_name, metric in result_dict.items():
    df = pd.DataFrame(metric)[METRICS]
    all_dfs.append(df.T)
pd.concat(all_dfs, axis=0).iloc[:, 10:]

,10,11,12,13,14,15,16,17,18,19
test/swa_acc,0.770600,0.774000,0.782300,0.789400,0.808200,0.809100,0.819900,0.834700,0.827000,0.841700
test/swa_nll,0.716440,0.691743,0.674876,0.641045,0.584050,0.577012,0.559762,0.518876,0.532278,0.510366
test/swa_ece,0.054092,0.047032,0.036750,0.025469,0.016743,0.014856,0.013409,0.017686,0.017217,0.037530


In [108]:
result_dict.keys()

odict_keys(['cifar10_snapshot_20220919_141947', 'cifar10_snapshot_20220919_141955', 'cifar10_snapshot_20220919_141959', 'cifar10_snapshot_20220919_142002', 'cifar10_snapshot_20220919_142006'])